In [52]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from string import punctuation
import numpy as np
import pandas as pd

In [53]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Парсинг
Парсим сайт с хорошими новостями про животных: http://goodnewsanimal.ru/
Ключевые слова представлены тегами, что очень удобно

In [56]:
news = 'http://goodnewsanimal.ru/'

In [57]:
def get_ss(page):
    news_list = []
    response = requests.get(page)
    soup = BeautifulSoup(response.text)
    code = soup.find_all('div', {'class': 'newsTitle'})
    for cod in code:
        a = cod.find('a').attrs['href']
        news_list.append(a)
    news_list_full = ['http://goodnewsanimal.ru' + new for new in news_list]
    return(news_list_full)


my_news = get_ss(news)
print(len(my_news))

18


In [58]:
def get_info(link):
    resp = requests.get(link)
    soup = BeautifulSoup(resp.text)
    details = soup.find('div', {'class': "details"}).text
    novost = soup.find('div', {'class': 'message'})
    ne = novost.find_all('p')
    text = []
    keywords = soup.find('meta', {'name': 'keywords'}).attrs['content'].split(', ')
    for n in ne:
        text.append(n.text)
    real_text = ' '.join(text)
    title = soup.find('title').text
    
    return real_text, title, keywords

Выберем несколько статей (я брала те, что подлиннее)

In [59]:
s = 0
for new in my_news[9:14]:
    text, title, keywords = get_info(new)
    s += len(text.split())
print(s) #общий размер корпуса в токенах 
my_news = my_news[9:14]

3099


In [60]:
all_texts = []
for new in my_news:
    text, title, kws = get_info(new)
    text = text.replace('(Хорошие новости про животных - goodnewsanimal.ru)', '')
    print(kws)
    print(text)
    all_texts.append(text)

['сон', 'спать', 'обезьяна', 'дельфин', 'косатка', 'рыба']
Многие люди думают, что животные спят точно также как и люди. Но это не так.  Некоторые животные вообще никогда полноценно не спят, но по другому дают отдохнуть своему мозгу. А насекомые, к примеру, просто впадают в что-то вроде оцепенения.  Даже человеческий сон для нас все еще очень загадочное явление, а уж сон животных и вовсе практически не изучен . 10. Как спят обезьяны  Человекоподобные приметы, такие как гориллы, орангутаны и шимпанзе, действительно спят почти как мы. На ночь они устраивают себе что-то вроде постели из листьев, травы или прутиков и сворачиваются в этих "гнездах" калачиком как люди в теплой постели. Мелкие приматы, вроде мартышек или макак, спят иначе, они просто замирают сидя на ветке в более удобной позе, шевелятся во сне и время от времени просыпаются, чтобы проверить, нет ли поблизости хищников.  9. Слоны и жирафы спят стоя  Крупные копытные животные вроде слонов и жирафов чаще всего просто не могут н

Я бы выделила следующие ключевые слова для каждой статьи:
1. сон, обезьяна, копытные, слон, дельфин, косатка, утка, рыба-попугай, морж
2. вомбат, еда, кража
3. лисята, лиса, настил, спасение
4. автомобиль, котенок, рысь, рысенок, автомобиль сбить
5. гулять, кот, собака, дом 

В качестве ключевых слов для задания я возьму объединение моих слов со словами из статьи

In [61]:
all_keywords = [['сон', 'обезьяна', 'копытный', 'дельфин', 'косатка', 'утка', 'рыба-попугай', 'рыба', 'морж'], 
                ['вомбат', 'еда', 'кража', 'Австралия'], 
                ['лисята', 'лиса', 'настил', 'спасение', 'Канада'], 
                ['котёнок', 'рысь', 'рысенок', 'автомобиль', 'автомобиль сбить'], 
                ['кот', 'кошка', 'гулять', 'собака', 'дом']]

# Автоматическое извлечение ключевых слов

в каждом методе для каждой из статей будем выделять топ-6 ключевых слов

## Для начала напишем функцию лемматизации текстов

In [62]:
m = MorphAnalyzer()
def lemmatize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if t not in punctuation:
            lemmas.append(m.parse(t)[0].normal_form)
    return ' '.join(lemmas)

## RAKE

In [63]:
rake = RAKE.Rake(stop)

минимальную частоту я выбираю в зависимости от длины статьи

In [64]:
rake_keywords = []
for new in all_texts:
    if len(new.split()) > 1000:
        minfreq = 3
    else:
        minfreq = 2
    print(lemmatize_text(new))
    print(rake.run(lemmatize_text(new), maxWords=2, minFrequency=minfreq)[:6])
    rake_keywords.append([a for (a, fr) in rake.run(lemmatize_text(new), maxWords=2, minFrequency=minfreq)])

многие человек думать что животное спать точно также как и человек но это не так некоторый животное вообще никогда полноценно не спать но по другой давать отдохнуть свой мозг а насекомое к пример просто впадать в что-то вроде оцепенение даже человеческий сон для мы всё ещё очень загадочный явление а уж сон животное и вовсе практически не изучить 10 как спать обезьяна человекоподобный примета такой как горилла орангутан и шимпанзе действительно спать почти как мы на ночь они устраивать себя что-то вроде постель из лист трава или прутик и сворачиваться в этот гнездо калачиком как человек в тёплый постель мелкий примат вроде мартышка или макака спать иначе они просто замирать сидя на ветка в более удобный поза шевелиться в сон и время от время просыпаться чтобы проверить нет ли поблизости хищник 9 слон и жираф спать стоя крупный копытный животное вроде слон и жираф частый весь просто не мочь найти себя надёжный укрытие для сон поэтому вынудить спать стоя чтобы всегда быть готовый к тот чт

## TextRank

In [67]:
from summa import keywords

In [69]:
textrank_keywords = []
for new in all_texts:
    print(keywords.keywords(lemmatize_text(new), language='russian', additional_stopwords=stop, scores=True)[:6])
    textrank_keywords.append([a for (a, fr) in list(keywords.keywords(lemmatize_text(new), language='russian', additional_stopwords=stop, scores=True))])

[('сон', 0.33905262357150545), ('рыба', 0.22444139462940038), ('животное спать', 0.2128423933190924), ('мочь', 0.20453640921989566), ('который', 0.18850938298104822), ('это', 0.1827450818140212)]
[('вомбат', 0.26952756011861934), ('центр помощь', 0.23792635649095495), ('мистер маг', 0.21528808973103047), ('вдоволь еда', 0.14329360858947016), ('несколько год', 0.1192534804047855), ('дикий природа', 0.11925348040478526)]
[('лиса', 0.32475642849937364), ('лис', 0.32475642849937364), ('малыш', 0.19456852091447804), ('брэд гейтс', 0.1863034902435629), ('маленький лисёнок', 0.17727725620088278), ('дом', 0.1766678503266947)]
[('котёнок', 0.4158896353000924), ('джилла хикс', 0.24422662849678123), ('очень', 0.16771242195644354), ('всё', 0.16141407431024976), ('это', 0.14657912080965343), ('животное', 0.1360111852948223)]
[('кот', 0.6146559480381711), ('эрина', 0.23598680028123006), ('домой', 0.2356693532728609), ('наш дом', 0.15180732698854654), ('грег', 0.14825642877946565), ('время', 0.141818

## collections.Counter



In [70]:
counter_keywords = []
for new in all_texts:
    words = [word for word in lemmatize_text(new).split() if word not in stop]
    print(Counter(words).most_common(20))
    counter_keywords.append([a for(a, fr) in Counter(words).most_common(20)])

[('сон', 30), ('спать', 27), ('мочь', 16), ('это', 12), ('который', 10), ('рыба', 9), ('птица', 8), ('животное', 7), ('фаза', 7), ('учёный', 7), ('человек', 6), ('хищник', 6), ('механизм', 6), ('быстрый', 6), ('утка', 6), ('морж', 6), ('просто', 5), ('весь', 5), ('минута', 5), ('дельфин', 5)]
[('центр', 11), ('вомбат', 8), ('мистер', 8), ('маг', 8), ('еда', 7), ('ещё', 3), ('это', 3), ('дикий', 3), ('сотрудник', 3), ('снова', 3), ('несколько', 2), ('маленький', 2), ('помощь', 2), ('приют', 2), ('вдоволь', 2), ('весь', 2), ('обратно', 2), ('природа', 2), ('запомнить', 2), ('также', 2)]
[('лисёнок', 6), ('гейтс', 5), ('лиса', 4), ('малыш', 4), ('настил', 4), ('маленький', 3), ('дом', 3), ('место', 3), ('ещё', 3), ('новый', 3), ('обнаружить', 2), ('крыльцо', 2), ('оставаться', 2), ('лишь', 2), ('наслать', 2), ('конец', 2), ('брэд', 2), ('друг', 2), ('лис', 2), ('неподалёку', 2)]
[('котёнок', 19), ('хикс', 13), ('это', 8), ('свой', 5), ('джилла', 4), ('очень', 4), ('ехать', 3), ('машина', 

## TF-IDF

In [71]:
vectorizer = TfidfVectorizer()
corp = []
for new in all_texts:
    corp.append(' '.join([word for word in lemmatize_text(new).split() if word not in stop]))
X = vectorizer.fit_transform(corp)

In [72]:
feature_names = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [73]:
tfidf_keywords = []
for i in range(5):
    words_fr = {}  
    feature_index = X[i,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [X[i, x] for x in feature_index])
    for w, s in [(feature_names[e], s) for (e, s) in tfidf_scores]:
        words_fr[w] = s
    sorted_tuples = sorted(words_fr.items(), key=lambda item: item[1])
    print(list(reversed(sorted_tuples))[:6]) 
    tfidf_keywords.append([a for (a, fr) in list(reversed(sorted_tuples))[:20]])

[('сон', 0.5260153179817011), ('спать', 0.47341378618353097), ('рыба', 0.22793997112540382), ('мочь', 0.18788191409359647), ('птица', 0.14027075146178694), ('фаза', 0.14027075146178694)]
[('центр', 0.4147292287380457), ('вомбат', 0.3738517739541633), ('мистер', 0.3738517739541633), ('маг', 0.3738517739541633), ('еда', 0.2190762788307068), ('сотрудник', 0.14019441523281126)]
[('лисёнок', 0.3686712093688333), ('лиса', 0.3686712093688333), ('гейтс', 0.3072260078073611), ('малыш', 0.24578080624588888), ('настил', 0.24578080624588888), ('место', 0.14872080518803807)]
[('котёнок', 0.6348418640852629), ('хикс', 0.43436548595307467), ('джилла', 0.1336509187547922), ('это', 0.12737075778738924), ('ехать', 0.10023818906609415), ('гараж', 0.10023818906609415)]
[('кот', 0.7120339270813718), ('эрина', 0.3215637090044905), ('дом', 0.18458996880123552), ('грег', 0.16078185450224525), ('собака', 0.12971778784753316), ('сидеть', 0.12971778784753316)]


## Соберём все данные в датафрейм

In [74]:
df = pd.DataFrame({'text': all_texts, 
                   'golden': all_keywords, 
                   'rake': rake_keywords, 
                   'textrank': textrank_keywords, 
                   'counter': counter_keywords, 
                   'tf-idf': tfidf_keywords})

In [75]:
df

,text,golden,rake,textrank,counter,tf-idf
0,"Многие люди думают, что животные спят точно та...","[сон, обезьяна, копытный, дельфин, косатка, ут...","[сон, спать, человек, земля, вода]","[сон, рыба, животное спать, мочь, который, это...","[сон, спать, мочь, это, который, рыба, птица, ...","[сон, спать, рыба, мочь, птица, фаза, учёный, ..."
1,"Несколько лет назад, когда вомбат по прозвищу ...","[вомбат, еда, кража, Австралия]","[еда, вомбат, ещё]","[вомбат, центр помощь, мистер маг, вдоволь еда...","[центр, вомбат, мистер, маг, еда, ещё, это, ди...","[центр, вомбат, мистер, маг, еда, сотрудник, с..."
2,В Канаде выводок из семи (!) маленьких лисят с...,"[лисята, лиса, настил, спасение, Канада]","[неподалёку, настил]","[лиса, лис, малыш, брэд гейтс, маленький лисён...","[лисёнок, гейтс, лиса, малыш, настил, маленьки...","[лисёнок, лиса, гейтс, малыш, настил, место, т..."
3,"Две недели назад Джилл Хикс из Читтануги, штат...","[котёнок, рысь, рысенок, автомобиль, автомобил...","[свой страничка, короткий хвост, котёнок, это,...","[котёнок, джилла хикс, очень, всё, это, животн...","[котёнок, хикс, это, свой, джилла, очень, ехат...","[котёнок, хикс, джилла, это, ехать, гараж, рыс..."
4,"Эрин Брин из Остина, штат Техас, впервые замет...","[кот, кошка, гулять, собака, дом]","[кот, собака, это, уходить, сидеть, гулять, см...","[кот, эрина, домой, наш дом, грег, время, день...","[кот, эрина, дом, это, собака, сидеть, хотеть,...","[кот, эрина, дом, грег, собака, сидеть, хотеть..."


# Применение фильтров

Мне ключевые слова представляются как только существительное/существительное + глагол. Попробуем найти эти два шаблона

In [76]:
templates = [['NOUN'], ['NOUN', 'VERB']]

In [77]:
def apply_template(keywords):
    all_pos = []
    right_keywords = []
    for kw in keywords:
        all_pos.append([m.parse(w)[0].tag.POS for w in kw.split()])
    for i, kw in enumerate(keywords):
        if all_pos[i] in templates:
            right_keywords.append(kw)
    return right_keywords

In [78]:
for i in ['rake', 'textrank', 'counter', 'tf-idf']:
    df[f'new_{i}'] = df[i].apply(apply_template)

df['new_golden'] = df['golden'].apply(apply_template)

In [79]:
df

,text,golden,rake,textrank,counter,tf-idf,new_rake,new_textrank,new_counter,new_tf-idf,new_golden
0,"Многие люди думают, что животные спят точно та...","[сон, обезьяна, копытный, дельфин, косатка, ут...","[сон, спать, человек, земля, вода]","[сон, рыба, животное спать, мочь, который, это...","[сон, спать, мочь, это, который, рыба, птица, ...","[сон, спать, рыба, мочь, птица, фаза, учёный, ...","[сон, человек, земля, вода]","[сон, рыба, мочь, учёный, морж, птица, детёныш...","[сон, мочь, рыба, птица, животное, фаза, учёны...","[сон, рыба, мочь, птица, фаза, учёный, хищник,...","[сон, обезьяна, дельфин, косатка, утка, рыба-п..."
1,"Несколько лет назад, когда вомбат по прозвищу ...","[вомбат, еда, кража, Австралия]","[еда, вомбат, ещё]","[вомбат, центр помощь, мистер маг, вдоволь еда...","[центр, вомбат, мистер, маг, еда, ещё, это, ди...","[центр, вомбат, мистер, маг, еда, сотрудник, с...","[еда, вомбат]","[вомбат, набег]","[центр, вомбат, мистер, маг, еда, сотрудник, п...","[центр, вомбат, мистер, маг, еда, сотрудник, п...","[вомбат, еда, кража, Австралия]"
2,В Канаде выводок из семи (!) маленьких лисят с...,"[лисята, лиса, настил, спасение, Канада]","[неподалёку, настил]","[лиса, лис, малыш, брэд гейтс, маленький лисён...","[лисёнок, гейтс, лиса, малыш, настил, маленьки...","[лисёнок, лиса, гейтс, малыш, настил, место, т...",[настил],"[лиса, лис, малыш, дом, место, настил, друг]","[лисёнок, гейтс, лиса, малыш, настил, дом, мес...","[лисёнок, лиса, гейтс, малыш, настил, место, д...","[лисята, лиса, настил, спасение, Канада]"
3,"Две недели назад Джилл Хикс из Читтануги, штат...","[котёнок, рысь, рысенок, автомобиль, автомобил...","[свой страничка, короткий хвост, котёнок, это,...","[котёнок, джилла хикс, очень, всё, это, животн...","[котёнок, хикс, это, свой, джилла, очень, ехат...","[котёнок, хикс, джилла, это, ехать, гараж, рыс...","[котёнок, гараж, друг, рука, дом, фейсбук, пос...","[котёнок, животное, рысь, рысёнок, женщина, со...","[котёнок, хикс, джилла, машина, друг, животное...","[котёнок, хикс, джилла, гараж, рысь, хвост, ры...","[котёнок, рысь, рысенок, автомобиль]"
4,"Эрин Брин из Остина, штат Техас, впервые замет...","[кот, кошка, гулять, собака, дом]","[кот, собака, это, уходить, сидеть, гулять, см...","[кот, эрина, домой, наш дом, грег, время, день...","[кот, эрина, дом, это, собака, сидеть, хотеть,...","[кот, эрина, дом, грег, собака, сидеть, хотеть...","[кот, собака, хозяин, фейсбук, крыльцо, диван,...","[кот, эрина, грег, время, макс, минута, майкл,...","[кот, эрина, дом, собака, грег, время, день, х...","[кот, эрина, дом, грег, собака, окно, время, х...","[кот, кошка, собака, дом]"


# Метрики

Перейдём к оценкам полноты, точности и F1-score
Напишем функцию, которая будет высчитывать параметр для каждого из методов (по столбцам датафрейма)

In [80]:
def evaluation(method):
    if method.startswith('new'):
        golden = df['new_golden']
    else:
        golden = df['golden']

    pred = df[method]

    precision = []
    recall = []
    f1_score = []

    for i in range(len(golden)):
        prec = np.intersect1d(golden[i], pred[i]).shape[0] / len(pred[i])
        rec = np.intersect1d(golden[i], pred[i]).shape[0] / len(golden[i])
   
        if prec + rec != 0:
            f1 = 2 * prec * rec / (prec + rec)
        else:
            f1 = 0
        precision.append(prec)
        recall.append(rec)
        f1_score.append(f1)

    return np.mean(precision), np.mean(recall), np.mean(f1_score)

In [86]:
methods = ['rake', 'textrank', 'tf-idf', 'counter', 'new_rake', 'new_textrank', 'new_tf-idf', 'new_counter']
metrics = {}
for m in methods:
    metrics[m] = evaluation(m)
df_res = pd.DataFrame(metrics).T
df_res.columns = ['precision', 'recall', 'f1-score']

In [87]:
df_res

,precision,recall,f1-score
rake,0.326218,0.322222,0.279365
textrank,0.074342,0.383333,0.122574
tf-idf,0.160000,0.571111,0.246299
counter,0.150000,0.531111,0.230299
new_rake,0.532143,0.315000,0.339394
new_textrank,0.247094,0.430000,0.271214
new_tf-idf,0.256620,0.605000,0.355952
new_counter,0.239976,0.555000,0.332279


# Анализ результатов

Сразу отмечу, что одной из главных проблем стала неполнота списка стоп-слов (в ключевых словах мелькают слова вроде *мочь*. 

Посмотрим на метрики. Лучшим результатом по F1-score стал отфильтрованный tf-idf, и в целом отфильтрованные результаты показали более высокиее результаты (неудивительно, потому что если я не выделялал сочетание глагол + глагол, то в ключевых словах я не ожидаю его увидеть). Самый высокий recall у tf-idf, то есть этот метод хорошо выделяет ключевые слова (но может достать ещё много того, что мы не ожидаем увидеть.

Precision лучше всего у rake - то есть этот метод лучше фильтрует не-ключевые слова. 

Для улучшения результатов я бы просто объединила несколько методов (например, если слово выделяется ключевым и в tf-ids и в rake, то шансов, что оно действительно ключевое, больше). 

ps хочу отметиь, что collections.Counter (то есть просто частота встречаемости) показывает очень хорошие результаты